EDA, Exploratory Data Analysis
===
Hyper-Parameter Optimization
---
As well-known fact, suitable tune parameter could get better result; here, we will introduce how to tune hyperparameters of machine learning algorithm in the most effective way.


1. nan conversion
2. target, $\mathbf{y\Rightarrow\log(1+y)}$ (`np.log1p`), for normalize fitting; later, back by $\mathbf{y_p \Rightarrow \exp(y_p)-1}$ (`np.expm1`); or use `np.log` directly if all the target data are greater than 0.
- latitude/longitude conversion, a°). knn means, b°). dbscans, then one-hot converstion
  ```Lasso, 0.7012 ➡︎ 0.6893```, the last one can not assign a fixed value to fix the data.
- different models, xgb, lgb, ...; here we try the `lightgbm`;
- stack model, blend moder, ...; install `mlxtend` by pip.
- <font color="red">Hyper-parameter tuning</font>: standard approaches such as Grid Search and Random Search.

Note
---
1. Since 2019/05/17, two new implementations of gradient boosting trees: `ensemble.HistGradientBoostingClassifier` and `ensemble.HistGradientBoostingRegressor` , are supported by `scikit_learning`-0.21.1.
```python
# usage
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor
HistGradientBoostingRegressor(loss=’least_squares’, learning_rate=0.1, max_iter=100, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20, l2_regularization=0.0, max_bins=256, scoring=None, validation_fraction=0.1, n_iter_no_change=None, tol=1e-07, verbose=0, random_state=None)
```


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm,tqdm_notebook
import folium

import seaborn as sns
%matplotlib inline

In [2]:
train_df = pd.read_csv('../dataset/train.csv')
test_df = pd.read_csv('../dataset/test.csv')

In [ ]:
print ("Train: ",train_df.shape[0],"sales, and ",train_df.shape[1],"features")
print ("Test:  ",test_df.shape[0],"sales, and ",test_df.shape[1],"features")

In [3]:
def fillnan(df_1,df_2,features):
    df_1[features[0]].fillna(value=0,inplace=True)
    df_2[features[0]].fillna(value=0,inplace=True)
    df_1[features[1]].fillna(value=0,inplace=True)
    df_2[features[1]].fillna(value=0,inplace=True)
    
    df=pd.concat([df_1[features],df_2[features]],axis=0)
    for f in features[2:]:
        df_1[f].fillna(value=df[f].median(),inplace=True)
        df_2[f].fillna(value=df[f].median(),inplace=True)
    return df_1,df_2

nan_features=['parking_area','parking_price','txn_floor','village_income_median']

In [4]:
# get rid of or replace nan
train_df,test_df=fillnan(train_df,test_df,nan_features)

Data
---
1. Quantitative
   - time-state: 'txn_dt', 'building_complete_dt'
   - non-time,
           'parking_price','building_area','village_income_median','town_population','town_area',
           'town_population_density','I_Min','II_MIN','III_MIN','IV_MIN','V_MIN','VI_MIN',
           'VII_MIN','VIII_MIN','IX_MIN','X_MIN','XI_MIN','XII_MIN','XIII_MIN','XIV_MIN',
   - location: 'lon','lat'
- Qualitative
  - building_material(9),city(11),total_floor(29),building_type(5),building_use(10),parking_way,
    parking_area, txn_floor,'doc_rate', 'master_rate', 'bachelor_rate', 'jobschool_rate',
       'highschool_rate', 'junior_rate', 'elementary_rate', 'born_rate',
       'death_rate', 'marriage_rate', 'divorce_rate'
  - village(2899)     

In [5]:
quantitative = ['txn_dt', 'building_complete_dt','parking_price','building_area','village_income_median','town_population','town_area',
           'town_population_density','I_MIN','II_MIN','III_MIN','IV_MIN','V_MIN','VI_MIN',
           'VII_MIN','VIII_MIN','IX_MIN','X_MIN','XI_MIN','XII_MIN','XIII_MIN','XIV_MIN',
           'lon','lat']
qualitative=['building_material','city','total_floor','building_type','building_use',
             'parking_way','parking_area','txn_floor','doc_rate', 'master_rate', 
             'bachelor_rate', 'jobschool_rate','highschool_rate', 'junior_rate', 
             'elementary_rate', 'born_rate','death_rate', 'marriage_rate', 'divorce_rate',
             'village']
target=['total_price']

In [6]:
train_df['total_price_log']=np.log(train_df['total_price'])

Skewness and Kurtosis
---
\begin{eqnarray}
    \text{Skewness }&=&\frac{E(X-\mu)^3}{\sigma^3}\\
    \text{Kurtosis }&=&\frac{E(X-\mu)^4}{\sigma^4}
\end{eqnarray}
1. Skewness $>0$, log-tail on the right-side, and $<0$, on the left side,
- Kurtosis large, steep in the peak.

Linear regression assumes that data should follow the `normal distributed` rule. Thus, we want to check the  quantitative features, continuous variables, whether obey the normal rule;
```
        convert the feature, with skew >1, by logarithm, ln(var);
        translate the data if data is smaller than 0.
```        

In [7]:
# check degree of skew 
skew_var_X={}
for i in range(len(quantitative)):
    skew_var_X[i]=abs(train_df[quantitative[i]].skew())
skew_df=pd.Series(skew_var_X)#.sort_values(ascending=False)
skew_df.index=quantitative
skew_df.sort_values(ascending=False)

building_area              83.710631
VIII_MIN                    8.002373
XIII_MIN                    7.606963
XII_MIN                     7.230123
XI_MIN                      7.088242
V_MIN                       5.753089
IX_MIN                      5.401486
parking_price               4.816163
X_MIN                       4.719582
VII_MIN                     4.632135
III_MIN                     4.372702
II_MIN                      4.077665
village_income_median       2.848078
I_MIN                       2.761933
XIV_MIN                     2.680964
IV_MIN                      2.428235
VI_MIN                      2.257881
town_area                   1.988336
lat                         1.220302
town_population_density     0.765919
lon                         0.764275
town_population             0.643185
txn_dt                      0.158794
building_complete_dt        0.030223
dtype: float64

In [8]:
train_df_skew=train_df.copy()
var_X_ln=skew_df.index[skew_df>1]
for i in var_X_ln:
    if min(train_df_skew[i])<=0:
       train_df_skew[i]=np.log(train_df_skew[i]+abs(train_df_skew[i].min())+0.01) 
    else:
       train_df_skew[i]=np.log(train_df_skew[i])

Lasso($\alpha$)
---
$$\text{arg min}\left(|| y-\sum\mathbf{\beta\cdot x}||+\alpha\sum|\beta_i|\right) $$

In [9]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

#Validation function
n_folds = 5

def rmsle_cv(model,X,y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse= np.sqrt(-cross_val_score(model, X.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

How to get the `better` alpha? Try one by one from $10^{-3}$ to 1.

In [12]:
# Set up train/valid data, X/y

X=train_df_skew.drop(['building_id','total_price','total_price_log'],axis=1)
y = train_df_skew['total_price_log']

In [14]:
from sklearn.linear_model import LassoCV

lasso_alphas = np.logspace(-3, 0, 100, base=10)
lcv = LassoCV(alphas=lasso_alphas, cv=10) # Search the min MSE by CV
lcv.fit(X, y)

print('The best alpha is {}'.format(lcv.alpha_))
print('The r-square is {}'.format(lcv.score(X, y))) 

The best alpha is 0.001
The r-square is 0.8917035438340241


In [ ]:
lcv.coef_

In [16]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0001, random_state=1,tol=0.01))

In [17]:
score = (rmsle_cv(lasso, X,y))
print("\nLasso score (alpha = {:.5f}): μ {:.4f} (𝝈 {:.4f})\n".format(0.0001,score.mean(), score.std()))


Lasso score (alpha = 0.00010): μ 0.3804 (𝝈 0.0036)



In [18]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.001, random_state=1,tol=0.01))

In [19]:
score = (rmsle_cv(lasso, X,y))
print("\nLasso score (alpha = {:.5f}): μ {:.4f} (𝝈 {:.4f})\n".format(0.0001,score.mean(), score.std()))


Lasso score (alpha = 0.00010): μ 0.3841 (𝝈 0.0036)



Practice 
---
Ridge Model ($\alpha$):
$$\text{arg min}\left(|| y-\sum\mathbf{\beta\cdot x}||+\alpha\sum|\beta_i|^2\right) $$

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-2, 3, 100, base=10)

# Search the min MSE by CV
rcv = RidgeCV(alphas=alphas, store_cv_values=True) 
rcv.fit(X, y)


print('The best alpha is {}'.format(rcv.alpha_))
print('The r-square is {}'.format(rcv.score(X, y))) 
# Default score is rsquared

In [ ]:
# Scoring train set 

In [21]:
def score_(y,yp):
    hit=sum(np.abs((y-yp))/y<0.1)/len(y)
    
    score=np.round(hit,4)*1e4+(1-np.round(hit,4))
    return score

In [26]:
clf=Lasso(alpha=0.0001)
clf.fit(X,y)

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
yp=clf.predict(X)

yp1=np.exp(yp)
y1=np.exp(y)
score_(y1,yp1)

2477.7523

Lightgbm, HyperParameter Tuning
---
We already watch out the power of lightgbm. Let ue to dig out more power by parameter tuning.

Here, conisder the following parameters:

- **Number of estimators**, number of boosting iterations, LightGBM is fairly robust to over-fitting so a large number usually results in better performance,
- **Maximum depth**,  limits the number of nodes in the tree, used to avoid overfitting ( max_depth=-1  means unlimited depth),
- **Number of leaves**, number of leaves in full tree,
- **Subsample**, each model trained at each iteration will have only a specific percentage subset of observations,
- **Column sampling by tree**, each model trained at each iteration will have only a specific percentage subset of features

In [39]:
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, train_test_split, \
                                     GridSearchCV, RandomizedSearchCV 


In [29]:
lightgbm = lgb.LGBMRegressor(objective='regression', 
                                       max_depth=8,
                                       num_leaves=63,
                                       subsample=0.8,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       colsample_bytree=0.8,
                                       feature_fraction=1,#0.2,
                                       feature_fraction_seed=7,
                                       verbose=1,
                                       )

In [30]:
score = rmsle_cv(lightgbm, X,y)
print("lightgbm: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

lightgbm: 0.2065 (0.0044)



In [31]:
lightgbm.fit(X,y)

yp=lightgbm.predict(X)

yp1=np.exp(yp)
y1=np.exp(y)

score_(y1,yp1)

6574.3426

In [42]:
hyper_space = {'n_estimators': [1000, 1500, 2000, 2500],
               'max_depth':  [4, 5, 8, -1],
               'num_leaves': [15, 31, 63, 127],
               'subsample': [0.6, 0.7, 0.8, 1.0],
               'colsample_bytree': [0.6, 0.7, 0.8, 1.0]}

In [36]:
# initialize the model
lgb_model = lgb.LGBMRegressor(boosting='gbdt', n_jobs=-1, random_state=2019)


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=2019)

In [ ]:
lgb_cv = GridSearchCV(lgb_model, hyper_space, scoring='r2', cv=5, verbose=1)
lgb_cv_results = lgb_cv.fit(X_train, y_train)
print("BEST PARAMETERS: " + str(lgb_cv_results.best_params_))
print("BEST CV SCORE: " + str(lgb_cv_results.best_score_))

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
# Predict (after fitting GridSearchCV is an estimator with best parameters)
y_pred = lgb_cv.predict(X_test)
 
# Score
score = r2_score(y_test, y_pred)
print("R2 SCORE ON TEST DATA: {}".format(score))

Practice
---

Complete the test stage and submit your score. 